# Pandas確認課題

このPandas確認問題は、データサイエンス100本ノックの問題で最低限必要な問題を抜粋したものですが、[Introduction_to_Pandas](./11_Introduction_to_Pandas.ipynb) に掲載されていない機能が必要な問題もあります。
初めて触るライブラリを調べながら使うというのはよくある光景です。この課題では皆さんにもそれに挑戦していただきます。  
ヒントとして検索キーワードなどを載せておくので、自力で調べながら解いてみましょう。  



## 必要モジュールのインポート

この問題で使うモジュールをインポートします．

In [78]:
import pandas as pd
import numpy as np

## データの読み込み

In [79]:
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

---
## 問1. 条件抽出
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [80]:
import pandas as pd

# データの読み込み
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

# 条件を満たすデータの抽出
filtered_df = df_receipt[
    (df_receipt['customer_id'] == 'CS018205000001') & 
    ((df_receipt['amount'] >= 1000) | (df_receipt['quantity'] >= 5))
]

# 必要な列の指定
result_df = filtered_df[['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']]

# インデックス番号の追加
result_df.reset_index(drop=True, inplace=True)


# 結果の表示
print(result_df)


   sales_ymd     customer_id  product_cd  quantity  amount
0   20180911  CS018205000001  P071401012         1    2200
1   20180414  CS018205000001  P060104007         6     600
2   20170614  CS018205000001  P050206001         5     990
3   20190226  CS018205000001  P071401020         1    2200
4   20180911  CS018205000001  P071401005         1    1100


---
## 問2. ソート
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭5件を全項目表示せよ。

In [81]:
import pandas as pd

# データの読み込み
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
# データフレームを作成
df_customer = pd.DataFrame(data)
# 生年月日で若い順にソート
df_customer_sorted = df_customer.sort_values(by='birth_day', ascending=False)

# 先頭5件を全項目表示
print(df_customer_sorted.head(5))


      customer_id customer_name  gender_cd gender   birth_day  age postal_cd  \
0  CS035114000004         大村 美里          1     女性  2007-11-25   11  156-0053   
1  CS022103000002        福山 はじめ          9     不明  2007-10-02   11  249-0006   
2  CS002113000009        柴田 真悠子          1     女性  2007-09-17   11  184-0014   
3  CS004115000014         松井 京子          1     女性  2007-08-09   11  165-0031   
4  CS002114000010          山内 遥          1     女性  2007-06-03   11  184-0015   

                 address application_store_cd application_date     status_cd  
0     東京都世田谷区桜**********               S13035         20150619  6-20091205-6  
1    神奈川県逗子市逗子**********               S14022         20160909  0-00000000-0  
2  東京都小金井市貫井南町**********               S13002         20160304  0-00000000-0  
3    東京都中野区上鷺宮**********               S13004         20161120  1-20081231-1  
4  東京都小金井市貫井北町**********               S13002         20160920  6-20100510-1  


---
## 問3. 全件数
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [82]:
import pandas as pd

# データフレームの読み込み
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

# 全体の件数をカウント
total_count = df_receipt.shape[0]

print(f"データフレームの件数: {total_count}")



データフレームの件数: 104681


## 問4. ユニーク件数
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [83]:
import pandas as pd

# データフレームの読み込み
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

# 顧客IDのユニークな件数をカウント
unique_customer_count = df_receipt['customer_id'].nunique()

print(f"ユニークな顧客IDの件数: {unique_customer_count}")



ユニークな顧客IDの件数: 8307


<details>
<summary>ヒント</summary>
「ユニーク」というのはそのまま検索に使える単語です。  
</details>

---
## 問5. 〇〇ごとに集計
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは先頭5件だけ表示せよ。

会員のみを抽出する方法は、例えば以下の2通りの方法があります。

In [84]:
df_receipt_only_member = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
df_receipt_only_member = df_receipt.query("not customer_id.str.startswith('Z')", engine="python")

In [85]:
import pandas as pd


# データの読み込み
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

# 非会員を除外
df_receipt_only_member = df_receipt.query("not customer_id.str.startswith('Z')", engine="python")

# 顧客IDごとに売上金額を合計
customer_amount_sum = df_receipt_only_member.groupby('customer_id')['amount'].sum()

# 全顧客の平均売上金額を計算
average_amount = customer_amount_sum.mean()

# 平均以上に買い物をしている顧客を抽出
customers_above_average = customer_amount_sum[customer_amount_sum >= average_amount]

# 先頭5件を表示
print(customers_above_average.head(5))


customer_id
CS001115000010    3044
CS001205000006    3337
CS001214000009    4685
CS001214000017    4132
CS001214000052    5639
Name: amount, dtype: int64


<details>
<summary>ヒント1</summary>
「pandas 要素ごと 集計」 などで今回使える機能に関する記事が見つかります。
</details>

<details>
<summary>ヒント2</summary>
メソッド名は "groupby" です。
</details>

---
## 問6. DataFrameの結合
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は先頭5件だけ表示せよ。

In [86]:
df_customer_only_member = df_customer[~df_customer["customer_id"].str.startswith("Z")]
df_customer_only_member = df_customer.query("not customer_id.str.startswith('Z')", engine="python")

In [87]:
import pandas as pd

# データフレームの読み込み
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

# 女性の顧客を対象とし、非会員（顧客IDが 'Z' から始まるもの）を除外
df_female_customers = df_customer[(df_customer['gender_cd'] == 1) & (~df_customer['customer_id'].str.startswith('Z'))]

# レシート明細の顧客IDと売上金額を集計
df_sales = df_receipt.groupby('customer_id')['amount'].sum().reset_index()

# 顧客IDが買い物の実績がない顧客については売上金額を0とするために左結合
df_customer_sales = pd.merge(df_female_customers[['customer_id']], df_sales, on='customer_id', how='left')

# 売上金額がNaNの顧客は0に置き換える
df_customer_sales['amount'].fillna(0, inplace=True)

# 結果を表示（先頭5件）
print(df_customer_sales.head())


      customer_id  amount
0  CS021313000114     0.0
1  CS031415000172  5088.0
2  CS028811000001     0.0
3  CS001215000145   875.0
4  CS015414000103  3122.0


/var/folders/g0/b8t31pt105q3qwtp75xv0srr0000gn/T/ipykernel_5755/4249959191.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_customer_sales['amount'].fillna(0, inplace=True)


<details>
<summary>ヒント1</summary>
タイトル通り 「pandas DataFrame 結合」などと調べれば必要な機能に関する記事が見つかります。  
</details>


<details>
<summary>ヒント2</summary>
"merge", "join"という似たメソッドがあります。  
今回の場合"merge"が便利でしょう。
</details>

---
## 問7. 時系列データ
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。なお、データは先頭5件を表示せよ。

In [88]:
import pandas as pd

# データフレームの読み込み
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')

# application_date 列を日付型に変換
df_customer['application_date'] = pd.to_datetime(df_customer['application_date'], format='%Y%m%d')

# 顧客IDと申し込み日を抽出
df_result = df_customer[['customer_id', 'application_date']]

# 結果を表示（先頭5件）
print(df_result.head())


      customer_id application_date
0  CS021313000114       2015-09-05
1  CS037613000071       2015-04-14
2  CS031415000172       2015-05-29
3  CS028811000001       2016-01-15
4  CS001215000145       2017-06-05


<details>
<summary>ヒント1</summary>
「pandas datetime」などで該当の機能が見つかるかと思います。
</details>


<details>
<summary>ヒント2</summary>
"pd.to_datetime"というメソッドが使えるでしょう。このメソッドを適用する際ですが、for文を使わずに実装しましょう。

---
## 問8. 関数
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、結果は先頭5件を表示せよ。

In [89]:
import pandas as pd
import numpy as np

# データフレームの読み込み
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

# 顧客IDが "Z" から始まるものを除外
df_filtered = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]

# 顧客IDごとに売上金額を合計
df_sum = df_filtered.groupby('customer_id')['amount'].sum().reset_index()

# 合計売上金額の常用対数を計算
df_sum['log_amount'] = np.log10(df_sum['amount'])

# 結果を表示（先頭5件）
print(df_sum.head())


      customer_id  amount  log_amount
0  CS001113000004    1298    3.113275
1  CS001114000005     626    2.796574
2  CS001115000010    3044    3.483445
3  CS001205000004    1988    3.298416
4  CS001205000006    3337    3.523356


---
## 問9. 欠損数
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [90]:
import pandas as pd
# データフレームの読み込み
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')

# 各項目の欠損値の数を確認
missing_counts = df_product.isnull().sum()

print("各項目の欠損値の数:")
print(missing_counts)


各項目の欠損値の数:
product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64


---
## 問10. 欠損値の除去
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [91]:
import pandas as pd

# データフレームの読み込み
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')

# 削除前の件数を表示
count_before = df_product.shape[0]
print(f"削除前の件数: {count_before}")

# 欠損値を含むレコードを削除
df_product_1 = df_product.dropna()

# 削除後の件数を表示
count_after = df_product_1.shape[0]
print(f"削除後の件数: {count_after}")

# 削除前後の件数の差を確認
deleted_count = count_before - count_after
print(f"削除されたレコード数: {deleted_count}")


削除前の件数: 10030
削除後の件数: 10023
削除されたレコード数: 7


In [92]:
len(df_product), len(df_product_1)

(10030, 10023)

---
## 問11. 欠損値の穴埋め
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とせよ。補完実施後、各項目について欠損が生じていないことも確認すること。

In [94]:
import pandas as pd

# データフレームの読み込み
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')

# 単価と原価の平均値を計算し、1円未満を四捨五入
average_unit_price = round(df_product['unit_price'].mean())
average_unit_cost = round(df_product['unit_cost'].mean())

# 欠損値を平均値で補完
df_product_2 = df_product.copy()
df_product_2['unit_price'].fillna(average_unit_price, inplace=True)
df_product_2['unit_cost'].fillna(average_unit_cost, inplace=True)

# 各項目について欠損が生じていないことを確認
print(df_product_2.isnull().sum())

# 結果の表示（確認のため先頭5件を表示）
print(df_product_2.head())


product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64
   product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   

   unit_price  unit_cost  
0       198.0      149.0  
1       218.0      164.0  
2       230.0      173.0  
3       248.0      186.0  
4       268.0      201.0  


/var/folders/g0/b8t31pt105q3qwtp75xv0srr0000gn/T/ipykernel_5755/3548470689.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_product_2['unit_price'].fillna(average_unit_price, inplace=True)
/var/folders/g0/b8t31pt105q3qwtp75xv0srr0000gn/T/ipykernel_5755/3548470689.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are 

### 余談
ChatGPTやBing AIに聞けば大抵のことは教えてくれます。  
何回か入力文章を吟味しないといけないこともありますが、知らないことを調べる場合は自分で検索するよりも早いです。  
ただ、ChatGPTなどは嘘をつく場合があるので、自分でソースを参照する姿勢は必要です。  

これはBingAIの回答例です。  

![BingAIの回答例](./imgs/pandas/BingAI.png)